In [1]:
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install folium



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# 1. 필요한 라이브러리 불러오기

import collections
import datetime
import glob
import math
import os
import pathlib

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import folium


pd.options.display.float_format = '{:,}'.format

plt.rcParams['font.family'] ='D2Coding'
plt.rcParams['axes.unicode_minus'] =False

DAY_OF_WEEK = ['월', '화', '수', '목', '금', '토', '일']

BASE_DIR = pathlib.Path('../').resolve()
print(os.listdir(BASE_DIR / '실습' / 'rawfiles'))

palette = sns.color_palette('husl', n_colors=7)
palette

['CARD_SUBWAY_MONTH_201903.csv', 'CARD_SUBWAY_MONTH_201902.csv', 'CARD_SUBWAY_MONTH_201901.csv', 'CARD_SUBWAY_MONTH_201905.csv', 'CARD_SUBWAY_MONTH_201904.csv', 'CARD_SUBWAY_MONTH_201906.csv', '지하철노선위경도정보2.csv']


[(0.9677975592919913, 0.44127456009157356, 0.5358103155058701),
 (0.7757319041862729, 0.5784925270759935, 0.19475566538551875),
 (0.5105309046900421, 0.6614299289084904, 0.1930849118538962),
 (0.20433460114757862, 0.6863857739476534, 0.5407103379425205),
 (0.21662978923073606, 0.6676586160122123, 0.7318695594345369),
 (0.5049017849530067, 0.5909119231215284, 0.9584657252128558),
 (0.9587050080494409, 0.3662259565791742, 0.9231469575614251)]

In [3]:
# 3. 데이터를 병합하여 하나의 데이터프레임 형성

df_subway_raw = pd.DataFrame()

for filename in glob.glob(str(BASE_DIR / '실습' / 'rawfiles' / 'CARD_SUBWAY_MONTH_*.csv')):
    tmp = pd.read_csv(filename)
    df_subway_raw = pd.concat([df_subway_raw, tmp])

df_subway_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107028 entries, 0 to 17723
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    107028 non-null  int64  
 1   노선명     107028 non-null  object 
 2   역명      107028 non-null  object 
 3   승차총승객수  107028 non-null  float64
 4   하차총승객수  107028 non-null  float64
 5   등록일자    107028 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 5.7+ MB


In [4]:
# dt.dayofweek로 표현된 요일의 번호가 0~6까지 각각 월~일에 대응되는지 확인해보자.

df = df_subway_raw.copy()

df['사용일자'] = pd.to_datetime(df['사용일자'], format='%Y%m%d')
df['요일'] = df['사용일자'].dt.dayofweek

# 달력을 통해, 2019년 5월 5일 어린이날은 '일요일'인 것을 알고있는 상황.
# 2019년도 어린이 날에 해당하는 데이터만 필터링한다.
df = df[df['사용일자'] == '2019-05-05']

# 어린이 날(일요일)의 dayofweek(숫자)를 확인한다.
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일
2370,2019-05-05,4호선,서울역,"45,856.0","42,422.0",20190508,6
2371,2019-05-05,2호선,시청,"15,457.0","15,413.0",20190508,6
2372,2019-05-05,1호선,종각,"31,735.0","27,154.0",20190508,6
2373,2019-05-05,3호선,종로3가,"26,047.0","24,846.0",20190508,6
2374,2019-05-05,1호선,종로5가,"16,105.0","16,292.0",20190508,6


In [5]:
df_subway_raw['사용일자'] = pd.to_datetime(df_subway_raw['사용일자'], format='%Y%m%d')
df_subway_raw['주차'] = df_subway_raw['사용일자'].dt.dayofyear // 7 + 1
df_subway_raw['요일'] = df_subway_raw['사용일자'].dt.dayofweek.map({ idx: val for idx, val in enumerate(DAY_OF_WEEK) })
df_subway_raw['연'] = df_subway_raw['사용일자'].dt.year
df_subway_raw['월'] = df_subway_raw['사용일자'].dt.month
df_subway_raw['일'] = df_subway_raw['사용일자'].dt.day
df_subway_raw['승하차총승객수'] = (df_subway_raw['승차총승객수'] + df_subway_raw['하차총승객수']).astype(int)

df_subway_raw.sample(10)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수
897,2019-01-02,경의선,공덕,"3,853.0","3,177.0",20190105,1,수,2019,1,2,7030
7468,2019-06-13,5호선,영등포시장,"11,310.0","11,716.0",20190616,24,목,2019,6,13,23026
10067,2019-02-18,공항철도,운서,"7,164.0","7,445.0",20190221,8,월,2019,2,18,14609
9545,2019-06-17,4호선,상계,"23,893.0","21,539.0",20190620,25,월,2019,6,17,45432
12433,2019-03-22,우이신설경전철,삼양,"2,353.0","2,526.0",20190325,12,금,2019,3,22,4879
12824,2019-04-22,5호선,군자,"12,377.0","13,515.0",20190425,17,월,2019,4,22,25892
5628,2019-03-10,2호선,홍대입구,"1,929.0","2,308.0",20190313,10,일,2019,3,10,4237
6110,2019-02-11,1호선,동두천,"2,204.0","2,578.0",20190214,7,월,2019,2,11,4782
788,2019-01-02,1호선,망월사,"8,115.0","8,198.0",20190105,1,수,2019,1,2,16313
3660,2019-03-07,4호선,숙대입구,"21,208.0","21,055.0",20190310,10,목,2019,3,7,42263


In [6]:
holidays = [
    datetime.date(2019, 1, 1),  # 신정
    datetime.date(2019, 2, 4),  # 설날(연휴)
    datetime.date(2019, 2, 5),  # 설날
    datetime.date(2019, 2, 6),  # 설날(연휴)
    datetime.date(2019, 3, 1),  # 삼일절
    datetime.date(2019, 5, 5),  # 어린이날
    datetime.date(2019, 5, 6),  # (어린이날)대체휴일
    datetime.date(2019, 5, 12),  # 부처님오신날
    datetime.date(2019, 6, 6),  # 현충일
    datetime.date(2019, 8, 15),  # 광복절
    datetime.date(2019, 9, 12),  # 추석(연휴)
    datetime.date(2019, 9, 13),  # 추석
    datetime.date(2019, 9, 14),  # 추석(연휴)
    datetime.date(2019, 10, 3),  # 개천절
    datetime.date(2019, 10, 9),  # 한글날
    datetime.date(2019, 12, 25),  # 크리스마스
]

df_subway_raw['공휴일'] = df_subway_raw['일'].isin(holidays)

df_subway_raw.sample()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수,공휴일
8651,2019-03-15,경강선,여주,"2,969.0","2,841.0",20190318,11,금,2019,3,15,5810,False


In [7]:
df_location_raw = pd.read_csv(str(BASE_DIR / '실습' / 'rawfiles' / '지하철노선위경도정보2.csv'))
df_location_raw['역명'] = df_location_raw['역이름']
del df_location_raw['역이름']
df_location_raw.sample()

,역지역,위도,경도,호선,역명
906,대구,35.85669699,128.5834577,3호선,남산


In [8]:
df_subway_raw = df_subway_raw.merge(df_location_raw, on='역명', how='left')
df_subway_raw.sample(5)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수,공휴일,역지역,위도,경도,호선
20348,2019-03-24,우이신설경전철,솔밭공원,"1,690.0","1,620.0",20190327,12,일,2019,3,24,3310,False,수도권,37.65597826,127.0132591,우이신설경전철
43029,2019-02-20,1호선,소사,"11,551.0","11,003.0",20190223,8,수,2019,2,20,22554,False,수도권,37.48279309,126.7956262,1호선
117854,2019-04-19,2호선,용두,"2,957.0","2,891.0",20190422,16,금,2019,4,19,5848,False,수도권,37.57405492,127.0382621,2호선
130499,2019-06-04,2호선,을지로4가,"14,761.0","14,601.0",20190607,23,화,2019,6,4,29362,False,수도권,37.5666369,126.997623,5호선
120147,2019-04-21,7호선,철산,"14,076.0","14,035.0",20190424,16,일,2019,4,21,28111,False,수도권,37.47619283,126.868202,7호선


In [9]:
df_subway_raw['노선명'] = df_subway_raw['호선']
del df_subway_raw['호선']
df_subway_raw.sample(5)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수,공휴일,역지역,위도,경도
73575,2019-01-28,분당선,복정,"12,029.0","9,841.0",20190131,5,월,2019,1,28,21870,False,수도권,37.47073829,127.1267014
8938,2019-03-11,경의선,강매,"3,521.0","2,918.0",20190314,11,월,2019,3,11,6439,False,수도권,37.61228686,126.8444948
18718,2019-03-23,분당선,왕십리,"19,412.0","16,817.0",20190326,12,토,2019,3,23,36229,False,수도권,37.56126116,127.0370785
130901,2019-06-04,1호선,온양온천,"5,862.0","5,875.0",20190607,23,화,2019,6,4,11737,False,수도권,36.78053767,127.0031815
75445,2019-01-31,2호선,삼성,"65,186.0","67,747.0",20190203,5,목,2019,1,31,132933,False,수도권,37.50881193,127.0630209


In [10]:
df = pd.read_csv(str(BASE_DIR / 'data' / '서울교통공사 역주소 및 전화번호_20240331.csv'))
df['지역구'] = df['도로명주소'].str.extract(r'\s(\w+구)\s')[0].str.strip()

df.sample(5)

,연번,역번호,호선,역명,역전화번호,Unnamed: 5,도로명주소,지번주소,지역구
20,21,211,2,성수,02-6110-2111,NaN,서울특별시 성동구 아차산로 100(성수동2가),서울특별시 성동구 성수동2가 300-1 성수역(2호선),성동구
170,171,2561,5,마천,02-6311-5601,NaN,서울특별시 송파구 마천로57길 지하7(마천동),서울특별시 송파구 마천동 307-4 마천역(5호선),송파구
281,282,4132,9,석촌고분,02-2656-0932,NaN,서울특별시 송파구 백제고분로 지하274(삼전동),서울특별시 송파구 삼전동 157-1 석촌고분역(9호선),송파구
188,189,2623,6,합정,02-6311-6221,NaN,서울특별시 마포구 양화로 지하45(합정동),서울특별시 마포구 합정동 414 합정역(6호선),마포구
73,74,322,3,동대입구,02-6110-3321,NaN,서울특별시 중구 동호로 지하256(장충동2가),서울특별시 중구 장충동2가 199-3 동대입구역(3호선),중구


In [11]:
# 기존 역 승하차 데이터에 지역구를 추가하자.

df_subway_raw = pd.merge(left=df_subway_raw, right=df[['역명', '지역구']], how='inner', on='역명')
df_subway_raw.sample(5)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,주차,요일,연,월,일,승하차총승객수,공휴일,역지역,위도,경도,지역구
89220,2019-06-21,경의선,공덕,"23,040.0","22,538.0",20190624,25,금,2019,6,21,45578,False,수도권,37.54453447,126.9514331,마포구
7304,2019-03-15,1호선,시청,"32,373.0","30,289.0",20190318,11,금,2019,3,15,62662,False,수도권,37.56533728,126.9771812,중구
19791,2019-02-08,7호선,고속터미널,"14,164.0","20,630.0",20190211,6,금,2019,2,8,34794,False,수도권,37.50453984,127.0051346,서초구
20228,2019-02-08,1호선,시청,"30,308.0","29,217.0",20190211,6,금,2019,2,8,59525,False,대전,36.35150604,127.3867335,중구
78462,2019-04-30,6호선,신당,"17,623.0","18,150.0",20190503,18,화,2019,4,30,35773,False,수도권,37.56566224,127.0194685,중구


In [12]:
df_subway_raw.to_csv(str(BASE_DIR / 'data' / '지하철_데이터병합_2019.csv'), index=False)